In [1]:
import pandas as pd

## Extract
___
En éste segmento leeremos el archivo y hacemos una limpieza básica de columnas y filas

In [227]:
df = pd.read_excel('HORARIOS(1)-convertido.xlsx')

In [229]:
df = df.iloc[6:,:]
df.rename(columns= {'Unnamed: 0':'Semestre','INSTITUTO POLITÉCNICO NACIONAL': 'Unidad de aprendizaje','Unnamed: 1':'Grupo','Unnamed: 2':'Clave','Unnamed: 4':'Academia','Unnamed: 5':'Lun','Unnamed: 6':'Mar','Unnamed: 7':'Mie','Unnamed: 8':'Jue','Unnamed: 9':'Vie','Unnamed: 10':'Profesor','Unnamed: 11':'Programa','Unnamed: 12':'Opcion','Unnamed: 13':'Departamento','Unnamed: 14':'Ocupabilidad'}, inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


## Web Scraping a las busquedas
___
 1. Haremos solicitudes de busqueda para cada maestro y obtenemos el link a misprofesores.com

In [231]:
import requests
from bs4 import BeautifulSoup

In [232]:
url_busqueda = 'https://www.google.com/search?&q='

In [233]:
diccionario = {}

In [234]:
def busqueda(nombre):
    global dicccionario
    if nombre not in diccionario.keys():
        request = requests.get(url_busqueda + nombre.lower().replace(' ','+'))
        
        if request.status_code == 200:
            parser = BeautifulSoup(request.text,'html.parser')
            direct_a = parser.find_all('a')
            for element in direct_a:
                if 'https://www.misprofesores.com/profesores/' in element.get('href'):
                    diccionario[nombre] = [element.get('href')[7:element.get('href').index('&')], get_grade(element.get('href')[7:element.get('href').index('&')])]
                    break
                else:
                    diccionario[nombre] = 'ERROR'
        else:
            diccionario[nombre] = 'ERROR STATUS CODE'
        
    return diccionario[nombre]

In [235]:
df['Link'] = df.loc[:,'Profesor'].apply(busqueda)

<ipython-input-235-c7ba40be4ec6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Link'] = df.loc[:,'Profesor'].apply(busqueda)


## Web scraping a la calificación
___
2. Ya teniendo el link, buscar la calificacion del maestro

In [191]:
def get_grade(link):
    request = requests.get(link)
    if request.status_code == 200:
        parser = BeautifulSoup(request.text,'html.parser')
        grade = parser.find('div',attrs={'class':'grade'})
        try:
            return grade.text
        except:
            return 'ERROR'

In [177]:
get_grade('https://www.misprofesores.com/profesores/FELIX-ZAMORA-SANCHEZ_42048')

'6.0'

In [203]:
df.loc[6,'Link']

['https://www.misprofesores.com/profesores/FELIX-ZAMORA-SANCHEZ_42048', '6.0']

In [207]:
def filtro1(lista):
    return lista[0]
def filtro2(lista):
    return lista[1]

In [249]:
FINAL = pd.DataFrame()
FINAL = df.iloc[:,:-1]
FINAL['Link'] = df.loc[:,'Link'].apply(filtro1)
FINAL['Calificacion'] = df.loc[:,'Link'].apply(filtro2)
FINAL.reset_index(inplace = True)
FINAL = FINAL.drop('index', axis = 1)

In [250]:
FINAL

,Semestre,Grupo,Clave,Unidad de aprendizaje,Academia,Lun,Mar,Mie,Jue,Vie,Profesor,Programa,Opcion,Departamento,Ocupabilidad,Link,Calificacion
0,1º,1MV2,M104,GEOMETRÍA ANALÍTICA,AG,16:30-\n18:00,NaN,16:30-\n18:00,NaN,16:30-\n18:00,FÉLIX ZAMORA SÁNCHEZ,LIM,TRONCO COMÚN,INGENIERÍA Y CIENCIAS SOCIALES,40,https://www.misprofesores.com/profesores/FELIX...,6.0
1,1º,1MV2,M101,SOCIEDAD Y CONOCLIMIENTO,CS,NaN,15:00-\n16:30,NaN,15:00-\n16:30,NaN,OSVALDO VILIULFO CÉSPEDES GAMA,LIM,TRONCO COMÚN,MATEMÁTICAS,40,https://www.misprofesores.com/profesores/osval...,5.3
2,1º,1MV2,M102,CÁLCULO I,CA,15:00-\n16:30,NaN,15:00-\n16:30,NaN,15:00-\n16:30,ISABEL HERNÁNDEZ TÉLLEZ,LIM,TRONCO COMÚN,MATEMÁTICAS,40,https://www.misprofesores.com/profesores/Isabe...,8.6
3,1º,1MV2,M105,INFORMÁTICA,I,18:00-\n19:30,NaN,18:00-\n19:30,18:00-\n19:30,NaN,ALFREDO GODÍNEZ MUÑOZ,LIM,TRONCO COMÚN,MATEMÁTICAS,40,https://www.misprofesores.com/profesores/Alfre...,3.5
4,1º,1MV2,M103,ÁLGEBRA I,AG,NaN,16:30-\n18:00,NaN,16:30-\n18:00,18:00-\n19:30,ALFONSO GARCÍA MURILLO,LIM,TRONCO COMÚN,INGENIERÍA Y CIENCIAS SOCIALES,40,https://www.misprofesores.com/profesores/ALFON...,4.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,8°,8FV1,836,ARQUITECTURA DE UNA COMPUTADORA,AG,16:30 -\n18:00,16:30 -\n18:00,NaN,16:30 -\n18:00,NaN,EDUARDO VIRUEÑA SILVA,LFM,MATEMÁTICAS,MATEMÁTICAS,30,https://www.misprofesores.com/profesores/Eduar...,7.4
187,8°,8FV2,867,INTRODUCCION A FÍSICA NUCLEAR Y PARTICULAS ELE...,F,15:00-\n17:00,NaN,15:00-\n17:00,NaN,15:00-\n17:00,MIGUEL NERI ROSAS,LFM,ESP. FÍSICA,FÍSICA,20,https://www.misprofesores.com/profesores/migue...,7.7
188,8º,8FV4,897,PROTECCIÓN RADIOLÓGICA,F,NaN,14:00-\n15:30,NaN,14:00-\n15:30,NaN,JULIÁN SÁNCHEZ GUTIÉRREZ,LFM,ING. NUCLEAR,FÍSICA,12,E,R
189,8º,8FV4,895,INGENIERÍA NUCLEAR II,F,15:00-\n16:30,NaN,15:00-\n16:30,NaN,NaN,JOSÉ AGUSTÍN FRANCISCO RODRÍGUEZ PÉREZ,LFM,ING. NUCLEAR,FÍSICA,12,E,R


In [252]:
FINAL.to_csv('Final.csv', index = False)